In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Load and preprocess data
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/archive/creditcard.csv')

# Split data into fraud and legitimate
fraud_data = df[df['Class'] == 1]
legit_data = df[df['Class'] == 0]

# Split both classes into train and test (80/20)
fraud_train, fraud_test = train_test_split(fraud_data, test_size=0.2, random_state=42)
legit_train, legit_test = train_test_split(legit_data, test_size=0.2, random_state=42)

# Preserve test set for final evaluation
final_test_set = pd.concat([fraud_test, legit_test])

# GAN will only train on fraud TRAINING data
fraud_train_features = fraud_train.drop(columns=['Class']).values

# Normalize data using scaler fitted ONLY on fraud training data
scaler = MinMaxScaler()
fraud_train_scaled = scaler.fit_transform(fraud_train_features)

# Convert to PyTorch dataset
real_data = torch.tensor(fraud_train_scaled, dtype=torch.float32)
dataset = TensorDataset(real_data)
data_loader = DataLoader(dataset, batch_size=128, shuffle=True, drop_last=True)

In [4]:
# Improved Generator with regularization
class Generator(nn.Module):
    def __init__(self, z_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2),

            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2),

            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2),

            nn.Linear(512, output_dim),
            nn.Sigmoid()
        )

    def forward(self, z):
        return self.model(z)

# Critic (Discriminator) for WGAN-GP
class Critic(nn.Module):
    def __init__(self, input_dim):
        super(Critic, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),

            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),

            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),

            nn.Linear(128, 1)
        )

    def forward(self, x):
        return self.model(x)

In [5]:
# Parameters
input_dim = real_data.shape[1]
z_dim = 30
n_critic = 5
lambda_gp = 10
epochs = 2000
batch_size = 128

# Initialize models and move to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = Generator(z_dim, input_dim).to(device)
critic = Critic(input_dim).to(device)

# Optimizers
optimizer_G = optim.Adam(generator.parameters(), lr=0.0001, betas=(0.5, 0.9))
optimizer_C = optim.Adam(critic.parameters(), lr=0.0001, betas=(0.5, 0.9))

In [6]:
# Gradient penalty for WGAN-GP
def compute_gradient_penalty(critic, real_samples, fake_samples):
    alpha = torch.rand(real_samples.size(0), 1).to(device)
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = critic(interpolates)

    gradients = torch.autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=torch.ones_like(d_interpolates),
        create_graph=True,
        retain_graph=True
    )[0]

    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

# Enable cuDNN auto-tuner for faster training
torch.backends.cudnn.benchmark = True

# Training loop
for epoch in range(epochs):
    for i, batch in enumerate(data_loader):
        real_batch = batch[0].to(device)

        # Train Critic (n_critic times)
        for _ in range(n_critic):
            # Generate fake data
            z = torch.randn(real_batch.size(0), z_dim, device=device)
            fake_batch = generator(z)

            # Compute critic scores
            real_validity = critic(real_batch)
            fake_validity = critic(fake_batch)

            # Gradient penalty
            gradient_penalty = compute_gradient_penalty(critic, real_batch.data, fake_batch.data)

            # Critic loss
            c_loss = -torch.mean(real_validity) + torch.mean(fake_validity) + lambda_gp * gradient_penalty

            optimizer_C.zero_grad()
            c_loss.backward()
            optimizer_C.step()

        # Train Generator
        z = torch.randn(real_batch.size(0), z_dim, device=device)
        gen_batch = generator(z)
        g_loss = -torch.mean(critic(gen_batch))

        optimizer_G.zero_grad()
        g_loss.backward()
        optimizer_G.step()

    # Print progress
    if epoch % 200 == 0:
        print(f"Epoch {epoch}/{epochs} | C_loss: {c_loss.item():.4f} | G_loss: {g_loss.item():.4f}")


/usr/local/lib/python3.11/dist-packages/torch/autograd/graph.py:823: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at /pytorch/aten/src/ATen/cuda/CublasHandlePool.cpp:180.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 0/2000 | C_loss: 6.8331 | G_loss: 0.0691
Epoch 200/2000 | C_loss: 0.0990 | G_loss: -0.2411
Epoch 400/2000 | C_loss: 0.1599 | G_loss: -0.2695
Epoch 600/2000 | C_loss: 0.1251 | G_loss: -0.6096
Epoch 800/2000 | C_loss: 0.0660 | G_loss: -0.6240
Epoch 1000/2000 | C_loss: 0.0959 | G_loss: -0.5625
Epoch 1200/2000 | C_loss: 0.0302 | G_loss: 0.5580
Epoch 1400/2000 | C_loss: -0.0135 | G_loss: 0.6501
Epoch 1600/2000 | C_loss: 0.1531 | G_loss: 0.5439
Epoch 1800/2000 | C_loss: 0.0593 | G_loss: 0.5019


In [7]:
# Generate synthetic fraud samples
generator.eval()
num_original_fraud_train = len(fraud_train)
num_legit_train = len(legit_train)
num_synthetic_needed = num_legit_train - num_original_fraud_train

synthetic_samples = []

with torch.no_grad():
    # Generate in batches
    for _ in range(0, num_synthetic_needed, batch_size):
        batch_size_ = min(batch_size, num_synthetic_needed - len(synthetic_samples))
        z = torch.randn(batch_size_, z_dim, device=device)
        gen_samples = generator(z).cpu().numpy()
        synthetic_samples.extend(gen_samples)

# Create synthetic fraud dataframe
synthetic_fraud = pd.DataFrame(synthetic_samples, columns=fraud_train.drop(columns=['Class']).columns)
synthetic_fraud['Class'] = 1

# Create balanced training set
balanced_train = pd.concat([
    fraud_train,  # Original fraud training data
    legit_train,   # Original legitimate training data
    synthetic_fraud  # Synthetic fraud
], ignore_index=True)

# Shuffle the dataset
balanced_train = balanced_train.sample(frac=1, random_state=42).reset_index(drop=True)

# Save datasets
balanced_train.to_csv('/content/drive/MyDrive/Colab Notebooks/archive/balanced_creditcard_train.csv', index=False)
final_test_set.to_csv('/content/drive/MyDrive/Colab Notebooks/archive/creditcard_test.csv', index=False)

print(f"Original fraud training samples: {len(fraud_train)}")
print(f"Synthetic fraud samples generated: {len(synthetic_fraud)}")
print(f"Balanced training set size: {len(balanced_train)}")
print(f"Test set size: {len(final_test_set)}")
print(f"Class distribution in training set:\n{balanced_train['Class'].value_counts()}")

Original fraud training samples: 393
Synthetic fraud samples generated: 227059
Balanced training set size: 454904
Test set size: 56962
Class distribution in training set:
Class
1    227452
0    227452
Name: count, dtype: int64


In [8]:

print(f"Original fraud training samples: {len(fraud_train)}")
print(f"Synthetic fraud samples generated: {len(synthetic_fraud)}")
print(f"Balanced training set size: {len(balanced_train)}")
print(f"Test set size: {len(final_test_set)}")
print(f"Class distribution in training set:\n{balanced_train['Class'].value_counts()}")
print(f"Class distribution in training set:\n{final_test_set['Class'].value_counts()}")

Original fraud training samples: 393
Synthetic fraud samples generated: 227059
Balanced training set size: 454904
Test set size: 56962
Class distribution in training set:
Class
1    227452
0    227452
Name: count, dtype: int64
Class distribution in training set:
Class
0    56863
1       99
Name: count, dtype: int64


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

def evaluate_model(model, x_train, y_train, x_legit_test, y_legit_test, x_fraud_test, y_fraud_test):
    """
    Evaluates a given model on training and test data.

    Args:
        model: The machine learning model to evaluate.
        x_train, y_train: Training data and labels.
        x_legit_test, y_legit_test: Test data and labels for legitimate transactions.
        x_fraud_test, y_fraud_test: Test data and labels for fraudulent transactions.

    Returns:
        A dictionary containing:
            - Legitimate accuracy
            - Fraud accuracy
    """
    # Train the model
    model.fit(x_train, y_train)

    # Evaluate accuracies
    accuracy_legit = model.score(x_legit_test, y_legit_test)
    accuracy_fraud = model.score(x_fraud_test, y_fraud_test)

    return {
        "accuracy_legit": accuracy_legit,
        "accuracy_fraud": accuracy_fraud
    }


models = {
    "Decision Tree": DecisionTreeClassifier(max_depth=4, random_state=42),
    "SVM": SVC(kernel='rbf', random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "AdaBoost": AdaBoostClassifier(n_estimators=50, random_state=42),
    "GaussianNB": GaussianNB(),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
}

In [ ]:
x_fraud_test = fraud_test[:, 1:30]
y_fraud_test = fraud_test[:, 30]

x_legit_test = legit_test[:, 1:30]
y_legit_test = legit_test[:, 30]

print('X_train Shape:', x_train.shape)
print('Y_train Shape:', y_train.shape)
print('X_fraud_test Shape:', x_fraud_test.shape)
print('Y_fraud_test Shape:', y_fraud_test.shape)
print('X_legit_test Shape:', x_legit_test.shape)
print('Y_legit_test Shape:', y_legit_test.shape)


In [ ]:
results = {}
for model_name, model in models.items():
    print(f"Evaluating {model_name}...")
    results[model_name] = evaluate_model(model, x_train, y_train, x_legit_test, y_legit_test, x_fraud_test, y_fraud_test)
    print(f"{model_name} Legit Accuracy: {results[model_name]['accuracy_legit']:.5f}")
    print(f"{model_name} Fraud Accuracy: {results[model_name]['accuracy_fraud']:.5f}")
